In [15]:
import qiskit

In [16]:
qiskit.__qiskit_version__

{'qiskit': '0.46.0', 'qiskit-aer': '0.13.3', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.20.2', 'qiskit-nature': '0.7.1', 'qiskit-finance': '0.4.0', 'qiskit-optimization': '0.6.0', 'qiskit-machine-learning': None}

In [17]:
import random

def generate_unique_random_ints(x, y):
    if x > y:
        raise ValueError("x must be less than or equal to y")
    random_ints = random.sample(range(y + 1), x)
    return random_ints


In [18]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer
from qiskit.circuit import Parameter

def generate_unique_random_pairs(n, x, qulist):
    entanglementpairlist = []
    for i in range(n):
        while True:
            random_control = random.randint(0, x)
            
            while True:
                if random_control in qulist:  #if control qubit is non-zero
                     break
                random_control = random.randint(0, x) #if zero, keep generate new control bit

            random_target = random.randint(0, x) #generate target bit
            qulist.append(random_target)
            while True:
                if random_target != random_control:
                    break
                random_target = random.randint(0, x) #generate a target which is not identical to the control

            entanglement = [random_control, random_target]
            
            pairexist = False
            for sublist in entanglementpairlist: #check whether this pair exists
                if sublist == entanglement:
                    pairexist = True
                    break

            if pairexist == False:
                entanglementpairlist.append(entanglement)
                break;

    return entanglementpairlist



In [44]:
 #quantum circuit with 5 qubits and 4 classical bits
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit_aer.primitives import Sampler

import random
from math import pi

N = 4
k = 3
l = 4
m = 3
t = 1 
t_overhead = 0.1 ##TODO: find toverhead
t_unit = 10 ** 5

qulist = []
u3list = []
entanglementlist = []
for j in range(l):
    random_integers = generate_unique_random_ints(k, 3) #randomly chose k in 0-3
    qulist += random_integers
    u3list += random_integers

    entanglementpair = generate_unique_random_pairs(m, 3, qulist)
    entanglementlist += entanglementpair


def buildqc(params):
    pos = 0
    u_pos = 0
    c_pos = 0
    
    qc = QuantumCircuit(N, N) #initialize circuit
    for j in range(l):
        for randomQubit in range(k):
            u1 = params[pos]
            pos += 1
            u2 = params[pos]
            pos += 1
            u3 = params[pos]
            pos += 1
            qc.u(u1, u2, u3, u3list[u_pos]) #add u3 gate
            u_pos += 1
        
        for randomPair in range(m):
            qc.cx(entanglementlist[c_pos][0], entanglementlist[c_pos][1])
            c_pos+=1 #create entanglement
    
        backend = Aer.get_backend('statevector_simulator')
        outputstate = backend.run(qc, shots = 1).result().get_statevector()
        probs = Statevector(outputstate).probabilities()
        ##print(probs)
        return probs


In [47]:
import numpy as np

def CFI(params):
    shift = pi/2
    num_runs = N * 2

    shift_list = []
    gradient_list = []

    """create list for shifted params"""
    for i in range(3 * k * l):
        shift_list.append([params[i] - pi/2, params[i] + pi/2])
        params_plus = params.copy()
        params_plus[i] = params[i] + pi/2
        params_minus = params.copy()
        params_minus[i] = params[i] - pi/2
        
        """run the circuit and store to the result list"""
        prob_minus = buildqc(params_minus)
        prob_plus = buildqc(params_plus)

        gradient_list.append(t ** 2 * (prob_plus - prob_minus)/2)

    CFI = []
    for i in range(N):
        CFI.append([])
        for j in range(N):
            CFI[i].append(gradient_list[i]*gradient_list[j])
    ##print(CFI)
    return CFI



    ##print(shift_list)
   
##CFI([1,2,3,4])

In [54]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.circuit import Parameter
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit.primitives import Sampler, Estimator

def objective_function(params):
    cfi_matrix = CFI(params)
    new_list = [[-element * t_unit * (t**2) / (t_overhead + t) for element in row] for row in cfi_matrix]
    return new_list


initial_params = []

for i in range(3 * k * l):
    initial_params.append(random.uniform(0, 2*np.pi))

# Choose the optimizer
# optimizer = Powell(maxiter=100)
optimizer = COBYLA(maxiter=500, tol=0.0001)

temp = initial_params.copy()
temp_fun = 0
# Run the optimization
for i in range(10):
    if (i == 0):
        result = optimizer.minimize(fun=objective_function, x0=initial_params)
        fun = -result.fun
        if (fun > temp_fun):
            temp_fun = fun
            temp = result.x.copy()
    else:
        result = optimizer.minimize(fun=objective_function, x0=temp)
        fun = -result.fun
        if (fun > temp_fun):
            temp_fun = fun
            temp = result.x.copy()

print(temp_fun)
    
    

22727.27271245041
